In this section, going over the problems I was not completely correct on in the fourth section of the problem set, DP.

> * house robber is O(N), not O(N**2) - you can use the combination skeleton from earlier to solve it, like so:


* Top-down brute force fast.

In [10]:
def rob(nums):
    if len(nums) == 0: return 0
    if len(nums) == 1: return nums[0]
    if len(nums) == 2: return max(*nums)
    return max(nums[0] + rob(nums[2:]), nums[1] + rob(nums[3:]))

In [11]:
rob([1,2,3,4,5])

9

In [12]:
rob([1,2])

2

In [13]:
rob([2,7,9,3,1])

12

* Now to memoize.

In [26]:
def rob(nums):
    def _rob(nums, memo):
        if len(nums) == 0: return 0, memo
        if len(nums) == 1: return nums[0], memo
        if tuple(nums) in memo:
            return memo[tuple(nums)], memo
        
        if len(nums) == 2:
            memo[tuple(nums)] = max(*nums)
            return memo[tuple(nums)], memo
        
        p1, memo = _rob(nums[2:], memo)
        p2, memo = _rob(nums[3:], memo)
        memo[tuple(nums)] = max(nums[0] + p1, nums[1] + p2)
        return memo[tuple(nums)], memo
    
    return _rob(nums, {})[0]

In [27]:
rob([2,7,9,3,1])

12

* This is $O(n)$ (this is the mistake I made last time) because memoized computations are constant work. In particular, we will solve `max([3, 1])`, then `max([5, 3, 1])`, then `max([9, 5, 3, 1])`...all of the subcalculations otherwise are memoized and constant work.

> https://leetcode.com/problems/house-robber-ii/

In [38]:
def rob(nums):
    if len(nums) == 0: return 0
    if len(nums) == 1: return nums[0]
    if len(nums) == 2: return max(*nums)
    return max(nums[0] + rob(nums[2:-1]), nums[1] + rob(nums[3:]))

In [39]:
rob([9, 5, 3, 1])

12

In [144]:
def rob(nums):
    if len(nums) == 0: return 0
    if len(nums) == 1: return nums[0]
    if len(nums) == 2: return max(nums[0], nums[1])
    
    def _rob(nums, memo):
        if len(nums) == 0: return 0, memo
        if len(nums) == 1: return nums[0], memo
        if tuple(nums) in memo:
            return memo[tuple(nums)], memo
        
        if len(nums) == 2:
            memo[tuple(nums)] = max(*nums)
            return memo[tuple(nums)], memo
        
        p1, memo = _rob(nums[2:], memo)
        p2, memo = _rob(nums[3:], memo)
        memo[tuple(nums)] = max(nums[0] + p1, nums[1] + p2)
        return memo[tuple(nums)], memo
    
    # ignore wrapping rule for non-final arrays
    # we always take one of the first entry, the second entry, or the last entry
    opt1, _ = _rob(nums[2:-1], {})
    opt2, _ = _rob(nums[1:-2], {})
    opt3, _ = _rob(nums[3:], {})
    
    return max(nums[0] + opt1, nums[-1] + opt2, nums[1] + opt3)

In [145]:
rob([1, 3, 1])

3

In [146]:
rob([1, 2, 3, 4, 5, 1, 2, 3, 4, 5])

16

In [147]:
rob([1, 3, 4, 1])

5

* Similarly $O(n)$, only difference is the need for three top-level cases.

> https://leetcode.com/problems/house-robber-iii/
>
> i'm noticing a pattern where your recursion sometimes acts on root.left.left, but ideally your recursion should only need to operate on root.left (the current node and not look forward twice - you had a similar issue with the base cases i mentioned earlier: your recursive call is accessing stuff that can be taken care of in another layer)

In [198]:
def rob(root):
    
    def grandchildren(root):
        out = []
        if root.left and root.left.left:
            out.append(root.left.left)
        if root.left and root.left.right:
            out.append(root.left.right)
        if root.right and root.right.left:
            out.append(root.right.left)
        if root.right and root.right.right:
            out.append(root.right.right)
        return out
    
    def _rob(root, memo):
        if not root:
            return 0, memo
        if not root.left and not root.right:
            return root.val, memo
        if root in memo:
            return memo[root], memo
        else:
            gchildren = grandchildren(root)
            gch_total_val = 0
            for gchild in gchildren:
                g_val, memo = _rob(gchild, memo)
                gch_total_val += g_val
            opt1 = root.val + gch_total_val
            opt2 = sum([_rob(root.left, memo)[0], _rob(root.right, memo)[0]])
            out = max(opt1, opt2)
            memo[root] = out
            return out, memo
    
    return _rob(root, {})[0]
    
class Node:
    def __init__(self, val):
        self.val = val
        self.left = None
        self.right = None

In [199]:
root = Node(1)
root.left = Node(2)
root.right = Node(3)
rob(root)

5

In [200]:
root = Node(10)
root.left = Node(2)
root.right = Node(3)
rob(root)

10

In [201]:
root = Node(1)
root.left = Node(2)
root.right = Node(3)
root.left.left = Node(1)
root.left.right = Node(1)
root.right.left = Node(1)
root.right.right = Node(1)
rob(root)

5

In [202]:
root = Node(1)
root.left = Node(2)
root.right = Node(3)
root.left.left = Node(2)
root.left.right = Node(2)
root.right.left = Node(2)
root.right.right = Node(2)
rob(root)

9

In [203]:
root = Node(1)
root.left = Node(2)
root.right = Node(3)
root.left.left = Node(2)
root.left.right = Node(2)
root.right.left = Node(2)
rob(root)

7

In [204]:
root = Node(2)
root.left = Node(1)
root.right = Node(3)
root.left.right = Node(4)
rob(root)

7

In [205]:
root = Node(1)
root.left = Node(2)
root.right = Node(6)
root.left.left = Node(2)
root.left.right = Node(2)
root.right.left = Node(2)
root.right.right = Node(2)
rob(root)

10

* Hmm, this problem seems to require looking at grandchildren as part of the recursive structure, though?

> https://leetcode.com/problems/minimum-cost-for-tickets/

* I can't redo this one because I remember the trick with it, because I was so irritated I missed it. :)

> https://leetcode.com/problems/edit-distance/

In [247]:
int(not ('a' == 'a'))

0

In [344]:
def minDistance(word1, word2):
    def findDistance(i, j):
        if i == 0 or j == 0:
            return max(i, j)
        if (i, j) in d:
            return d[(i, j)]
        
        d[(i - 1, j)] = findDistance(i - 1, j)
        case1 = d[(i - 1, j)] + 1
        d[(i, j - 1)] = findDistance(i, j - 1)
        case2 = d[(i, j - 1)] + 1
        d[(i - 1, j - 1)] = findDistance(i - 1, j - 1)
        case3 = d[(i - 1, j - 1)] + int(not (word1[i - 1] == word2[j - 1]))
        return min(case1, case2, case3)

    i = 1
    j = 1
    d = {(0, 0): 0, (0, 1): 1, (1, 0): 1}
    
    while i <= len(word1) and j <= len(word2):
        d[(i, j)] = findDistance(i, j)
        if j < i:
            j += 1
        else:
            i += 1
    
    i = min(i, len(word1))
    j = min(j, len(word2))
    while i <= len(word1):
        d[(i, j)] = findDistance(i, j)
        i += 1
    i -= 1
    while j <= len(word2):
        d[(i, j)] = findDistance(i, j)
        j += 1
    
    return d[(len(word1), len(word2))]

In [345]:
minDistance('a', 'a')

0

In [346]:
minDistance('abc', 'a')

2

In [347]:
minDistance('abc', 'abd')

1

In [348]:
minDistance('abc', 'abdgggg')

5

In [349]:
minDistance('b', 'c')

1

In [350]:
minDistance('101', '11')

1

In [351]:
minDistance('', '')

0

In [352]:
minDistance('', '101')

3

In [353]:
minDistance('dinitrophenylhydrazine', 'acetylphenylhydrazine')

6

* This is a bottom-up implementation using the Levenshtein distance recurrence relationship, which is not something I was able to derive independently.
* This algorithm is $O(n)$. Every combination of character counts where $|i - k| <= 1$ is visited. We then increment up to $\max(l(\text{word1}), l(\text{word2}))$, performing constant work along the way.
* I found the left-right incrementation tricky to implement.